In [17]:
%reload_ext autoreload
%autoreload 2

In [59]:
import pandas as pd
from dataclasses import dataclass
import beam_bearing_module as bbm

In [72]:
node_df = pd.read_csv("bearing data.csv", index_col=0)

In [73]:
node_df

,B1 Width,B1 Depth,B1 Route Length,B1 DL,B1 LL,B2 Width,B2 Depth,B2 Route Length,B2 DL,B2 LL,C Width,C Depth,F_c_perp,FRR
Label,,,,,,,,,,,,,,
BC1,10,20,4.0,8000,6000,8,18,3.5,8000,6000,12,12,400,2
BC2,8,20,5.0,8000,6000,9,18,4.0,8000,6000,12,12,400,2
BC3,8,20,4.5,8000,6000,10,18,5.0,8000,6000,12,12,400,2


In [74]:
def run_check(node_df):
    BearingNode = bbm.BearingNode(
        node_df['B1 Width'],
        node_df['B1 Depth'],
        node_df['B1 DL'],
        node_df['B1 LL'],
        node_df['B1 Route Length'],
        node_df['B2 Width'],
        node_df['B2 Depth'],
        node_df['B2 DL'],
        node_df['B2 LL'],
        node_df['B2 Route Length'],
        node_df['C Width'],
        node_df['C Depth'],
        node_df['F_c_perp'],
        3.2
        )

    b1_nonfire, b2_nonfire, b1_fire, b2_fire, loads = BearingNode.call_calculation()

    return b1_nonfire, b2_nonfire, b1_fire, b2_fire, loads 

In [75]:
df_capacity = node_df.apply(run_check, axis=1)
df_capacity

#df_output['Capacity'] = node_df.loc[:, 'FRR'].apply(run_check, axis=1)
#mini_df['category'] = mini_df.loc[:, 'gamma'].apply(categorize_concrete, threshold=2000)

Label
BC1    ((24048.0, 10.0, 4.0), (16834.0, 8.0, 3.5), (2...
BC2    ((24048.0, 8.0, 5.0), (21643.0, 9.0, 4.0), (60...
BC3    ((21643.0, 8.0, 4.5), (30060.0, 10.0, 5.0), (4...
dtype: object

In [76]:
'''

not sure this is the best way to do things but its how we did it

df_capacity[0]

((24048.0, 10.0, 4.0),  <--- b1 nonfire capacity, width, length
 (16834.0, 8.0, 3.5),   <--- b2 nonfire capacity, width, length
 (2693.0, 5.6, 0.7999999999999998), <--- b1 fire capacity, width, length
 (1010.0, 5.6, 0.2999999999999998), <--- b2 fire capacity, width, length
 (19200.0, 14000.0, 19200.0, 14000.0)) <--- b1fac unfac, b2fac unfac
 '''

'\n\nnot sure this is the best way to do things but its how we did it\n\ndf_capacity[0]\n\n((24048.0, 10.0, 4.0),  <--- b1 nonfire capacity, width, length\n (16834.0, 8.0, 3.5),   <--- b2 nonfire capacity, width, length\n (2693.0, 5.6, 0.7999999999999998), <--- b1 fire capacity, width, length\n (1010.0, 5.6, 0.2999999999999998), <--- b2 fire capacity, width, length\n (19200.0, 14000.0, 19200.0, 14000.0)) <--- b1fac unfac, b2fac unfac\n '

In [88]:
output_df = node_df.copy()

output_df.drop(output_df.columns[[3,4,8,9]], axis=1, inplace=True)

b1_nonfire_capacity = []
b2_nonfire_capacity = []

for row in df_capacity:
    b1_nonfire_capacity.append(row[0][0])
    b2_nonfire_capacity.append(row[1][0])

output_df.insert(3, "B1 Nonfire Case Capacity", b1_nonfire_capacity, True)
output_df.insert(11, "B2 Nonfire Case Capacity", b2_nonfire_capacity, True)


In [89]:
node_df

,B1 Width,B1 Depth,B1 Route Length,B1 DL,B1 LL,B2 Width,B2 Depth,B2 Route Length,B2 DL,B2 LL,C Width,C Depth,F_c_perp,FRR
Label,,,,,,,,,,,,,,
BC1,10,20,4.0,8000,6000,8,18,3.5,8000,6000,12,12,400,2
BC2,8,20,5.0,8000,6000,9,18,4.0,8000,6000,12,12,400,2
BC3,8,20,4.5,8000,6000,10,18,5.0,8000,6000,12,12,400,2


In [90]:
output_df

,B1 Width,B1 Depth,B1 Route Length,B1 Nonfire Case Capacity,B2 Width,B2 Depth,B2 Route Length,C Width,C Depth,F_c_perp,FRR,B2 Nonfire Case Capacity
Label,,,,,,,,,,,,
BC1,10,20,4.0,24048.0,8,18,3.5,12,12,400,2,16834.0
BC2,8,20,5.0,24048.0,9,18,4.0,12,12,400,2,21643.0
BC3,8,20,4.5,21643.0,10,18,5.0,12,12,400,2,30060.0


In [ ]:
  '''
    
    nf.text(-column_width*0.6,column_depth*1.9, ('Non-fire Case'), fontsize = 5, ha = 'center')
    nf.text(column_width/2, column_depth*1.75, ("Beam 1 Effective Bearing Area = " + str(round(b1_nonfire_width,3)) + " inch wide x " + str(round(b1_nonfire_length,3)) + " inch long."), fontsize = 5, ha = 'center')
    nf.text(column_width/2, -column_depth*0.8, ("Beam 2 Effective Bearing Area = " + str(round(b2_nonfire_width,3)) + " inch wide x " + str(round(b2_nonfire_length,3)) + " inch long."), fontsize = 5, ha = 'center')






not sure what this is doing but might be improtant!!!
if b1_fire[0] == 0:
    ratios.append(100000)
else:
    ratios.append(round(loads[1]/b1_fire[0],3))
    
ratios.append(round(loads[2]/b2_nonfire[0],3))

if b2_fire[0] == 0:
    ratios.append(100000)
else:
    ratios.append(round(loads[3]/b2_fire[0],3))

# NONFIRE CASE
# bx_nonfire = capacity, bearing width, routing length
# loads = b1_f, b1_u, b2_f, b2_u


--------

with col1:
    if beam1_route_length + beam2_route_length > column_depth:
        st.write('Your beam routing is overlapping. Please reduce one side.')

    if ratios[0] > 1:
        st.write("Non-fire capacity is " + str(b1_nonfire[0]) + " lbs. Demand is " + str(loads[0]) + " lbs. Ratio = " + str(ratios[0]) + ". FAIL!")
    else:
        st.write("Non-fire capacity is " + str(b1_nonfire[0]) + " lbs. Demand is " + str(loads[0]) + " lbs. Ratio = " + str(ratios[0]) + ". PASS!")
    if ratios[2] > 1:
        st.write("Non-fire capacity is " + str(b2_nonfire[0]) + " lbs. Demand is " + str(loads[2]) + " lbs. Ratio = " + str(ratios[2]) + ". FAIL!")
    else:
        st.write("Non-fire capacity is " + str(b2_nonfire[0]) + " lbs. Demand is " + str(loads[2]) + " lbs. Ratio = " + str(ratios[2]) + ". PASS!")
    fig


# FIRE CASE

if column_width - char_depth*2 >= beam1_width:
    b1_f_rect_x_pos = column_width/2-beam1_width/2
else:
    b1_f_rect_x_pos = char_depth

if column_width - char_depth*2 >= beam2_width:
    b2_f_rect_x_pos = column_width/2-beam2_width/2
else:
    b2_f_rect_x_pos = char_depth


#repeated code here because it doesn't let you put the same shape on two figures
column_outline = Rectangle([0, 0], column_width, column_depth, angle=0, facecolor='peachpuff', edgecolor='black')
beam1_outline = Rectangle([column_width/2-beam1_width/2, column_depth-beam1_route_length], beam1_width, column_depth*0.8, angle=0, facecolor='green', edgecolor='black')
beam2_outline = Rectangle([column_width/2-beam2_width/2, beam2_route_length], beam2_width, -column_depth*0.8, angle=0, facecolor='blue', edgecolor='black')
column_outline2 = Rectangle([0, 0], column_width, column_depth, angle=0, facecolor='peachpuff', edgecolor='black', fill=False, ls='--')

b1_f_bearing_area = Rectangle([b1_f_rect_x_pos, column_depth-beam1_route_length],b1_fire[1],b1_fire[2], angle=0, facecolor='magenta', edgecolor='black')
b2_f_bearing_area = Rectangle([b2_f_rect_x_pos, char_depth],b2_fire[1],b2_fire[2], angle=0, facecolor='magenta', edgecolor='black')

column_char_outline = Rectangle([char_depth,char_depth], column_width - char_depth*2, column_depth-char_depth*2, angle=0, edgecolor='red', fill = False)

fig = Figure()
f = fig.gca()

f.add_patch(column_outline)
f.add_patch(beam1_outline)
f.add_patch(beam2_outline)
f.add_patch(column_outline2)

f.add_patch(b1_f_bearing_area)
f.add_patch(b2_f_bearing_area)

f.add_patch(column_char_outline)

f.set_xlim(-column_width, column_width * 2)
f.set_ylim(-column_depth, column_depth * 2)
f.set_aspect('equal')

f.text(-column_width*0.6,column_depth*1.9, (FRR + ' Fire Case'), fontsize = 5, ha = 'center')
f.text(column_width/2, column_depth*1.75, ("Beam 1 Effective Bearing Area = " + str(round(b1_fire[1],3)) + " inch wide x " + str(round(b1_fire[2],3)) + " inch long."), fontsize = 5, ha = 'center')
f.text(column_width/2, -column_depth*0.8, ("Beam 2 Effective Bearing Area = " + str(round(b2_fire[1],3)) + " inch wide x " + str(round(b2_fire[2],3)) + " inch long."), fontsize = 5, ha = 'center')


with col2:
    if ratios[1] == 100000:
        st.write("After " + FRR + " fire, beam 1 has no bearing material remaining. Increase bearing length.")
    elif ratios[1] > 1:
        st.write(FRR + " fire capacity is " + str(b1_fire[0]) + " lbs. Demand is " + str(loads[1]) + " lbs. Ratio = " + str(ratios[1]) + ". FAIL!")
    else:
        st.write(FRR + " fire capacity is " + str(b1_fire[0]) + " lbs. Demand is " + str(loads[1]) + " lbs. Ratio = " + str(ratios[1]) + ". PASS!")
    if ratios[3] == 100000:
        st.write("After " + FRR + " fire, beam 2 has no bearing material remaining. Increase bearing length.")
    elif ratios[3] > 1:
        st.write(FRR + " fire capacity is " + str(b2_fire[0]) + " lbs. Demand is " + str(loads[3]) + " lbs. Ratio = " + str(ratios[3]) + ". FAIL!")
    else:
        st.write(FRR + " fire capacity is " + str(b2_fire[0]) + " lbs. Demand is " + str(loads[3]) + " lbs. Ratio = " + str(ratios[3]) + ". PASS!")
    fig



'''